In [79]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from matplotlib_scalebar.scalebar import ScaleBar

In [62]:
boundaries = gpd.read_file('../data/processed_data/city_boundary/Delft.gpkg').to_crs('EPSG:28992')

In [16]:
spatial_units = gpd.read_file('../data/processed_data/zones_delineation/PC_Delft.gpkg')
spatial_units = spatial_units.drop(columns = ['Postcode','gemeente','area_ratio'])

In [19]:
exposure = pd.read_csv('../data/results/exposure/Delft_exposure.csv', usecols= ['id_unit','expos_NW'])

In [22]:
spatial_units = spatial_units.merge(exposure, on = 'id_unit')
average_expos = spatial_units['expos_NW'].mean()
std_expos = spatial_units['expos_NW'].std()

In [29]:
regions = gpd.read_file('../data/results/regions/regions.gpkg')

In [31]:
regions_city = regions.loc[(regions['seg'] == 1) & (regions['city'] == 'Delft')]

In [40]:
spatial_units['test'] = 0
spatial_units['test'] = spatial_units['test'].mask(,(spatial_units['expos_NW']-average_expos)/std_expos)

In [82]:
fig, ax = plt.subplots(1, figsize=(8, 8))
#fig.set_tight_layout(True)
ax.margins(0)
spatial_units.plot(ax=ax, column = 'test', cmap = 'gnuplot_r') #cax to put the legend on a specific axis.
regions_city.plot(ax=ax,color = 'none',hatch='\\\\')
#boundaries.plot(ax=ax,color = 'none', edgecolor = 'black')
#df_zones.plot(ax=ax, color  = 'none', edgecolor='black')
ax.axis('off')
ax.add_artist(ScaleBar(1))
#plt.show()
fig.savefig('../report/figures/all_cities/Delft.png')
plt.close()